In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime

In [3]:
import methods
import formats
import json
import os
import re

from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_MISTRAL(calendar, tasks_list)
now = methods.current_time()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=37160277195-j0akooepafj0jqv9hbad9baeeoti72a5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52617%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Ftasks&state=zUNvtzxD2INrkVVDTM0aENSmIKxty6&access_type=offline
Based on the provided data, here are the events and to-do list for today (April 18, 2024):

Events for Today (April 18, 2024):
1. Team Meeting from 12:00 PM to 02:00 PM

To-Do List:
1. Title: 545 poster, Due Date: April 18, 2024

Please note that some of your to-do items have due dates in the past. You might want to review and update them accordingly. Also, the events and to-do list are provided based on the data given, and it's essential to double-check your calendar and tasks for accuracy.


In [4]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [5]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

In [7]:
def log_activity(role, message):
    filename = f"activity_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [11]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {calendar}\n To-Do Tasks: {tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my trusted virtual assistant, you are an expert in adeptly managing my calendar and assisting with scheduling tasks. Here is what I need from you:

1. **Calendar Management**:
 You have the capability to create, adjust, oversee, list, update, delete, and handle recurring events on my calendar as per my requests.
    - In situations where I don't provide enough information, you can ask for more details, upto 4 times, otherwise use your best judgement.
    - Should an event require scheduling or rescheduling and I haven't provided a specific time, consult my calendar and deduce a suitable time slot using your best judgement. Consider my usual habits, such as meal and sleep times, and assume a standard event duration of 1 hour unless your judgement suggests otherwise based on the nature of the event.
    - Before confirming any scheduling actions, scrutinize for any possible scheduling conflicts.
    - Always ask for confirmation before making any changes to the calendar. If I reject the suggested time, offer an alternative time slot.
    - Inform me when you have successfully completed your task and ensure there are no more pending queries from your end, otherwise always end with a question mark.

2. **Task Scheduling**:
You have the capability to schedule tasks from my To-Do list into my calendar.
    - Utilize your best judgement to organize and propose times for scheduling tasks listed in my To-Do list. Prioritize tasks based on urgency and balance throughout the available slots in my schedule.
    - Verify the calendar to ensure a task isn't already scheduled before adding it.
    - Check for any conflicts that may arise due to overlapping tasks or events.
    - Present a thoughtful plan for the scheduled tasks. Use your best judgement to predict task durations based on the nature of each task.
    - Ask for my confirmation before finalizing the schedule.
    - If I disagree with the proposed times, offer alternatives based on my feedback.
    - Always end your messages with a question mark to indicate that you are seeking a response from me, unless it's a confirmation of a task completion.

For example, if I request: 'Schedule a meeting with the design team on Wednesday,' your response might be: 'Based on your current calendar, Wednesday 10 AM is available and seems like a quiet time for you. Should I schedule the meeting for then?'
Example request: 'schedule all my tasks'
Example response: 'Sure! Here's a proposed schedule for all your tasks. Is there anything you'd like to change?
- Task 1: Apr 2nd, 10:00 AM - 11:00 AM
- Task 2: Apr 2nd, 11:00 AM - 12:00 PM
- Task 3: Apr 2nd, 12:00 PM - 1:00 PM
Please perform the necessary actions using the Google Calendar API. When you have completed my request, please do not end with a question mark and stop running. \n
Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""
# initial setup

initial_prompt = context + instruction +past_conversation

In [12]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request

In [13]:
messages = [
        ChatMessage(
            role="system",
            content=initial_prompt  # System message with instructions and context
        ),
        ChatMessage(
            role="user",
            content=user_request  # User's initial request
        )
    ]

log_activity("system", context+instruction)
log_activity("user", user_request)

log_message("user", user_request)


In [14]:
finalized_response = False

while not finalized_response:
    response = client.chat(model="mistral-large-latest", messages=messages)
    print(response.choices[0].message.content)
    log_activity("assistant", response.choices[0].message.content)
    log_message("assistant", response.choices[0].message.content)
    messages.append(ChatMessage(
        role="assistant",
        content=response.choices[0].message.content
    ))

    if re.compile(r'\?').search(response.choices[0].message.content):
        # If there is a question, do not finalize the response.
        # Wait for user feedback, then continue the loop.
        user_feedback = input()
        messages.append(ChatMessage(
            role="user",
            content=user_feedback
        ))
        log_message("user", user_feedback)
        log_activity("user", user_feedback)

    else:
        # If there is no question, finalize the response.
        finalized_response = True

Based on your current calendar, I see that you have some free time on the following days:

* April 23rd, after your Golf Practice (between 12:30 PM and 11:59 PM)
* April 24th, before your Senior Pics (between 12 AM and 3:30 PM)

Considering that jogging is usually done during daylight hours, I suggest scheduling your jog on April 23rd at 2:00 PM. This will give you ample time to rest after your Golf Practice and it falls within a suitable time frame for outdoor activities.

Would you like me to schedule the jog for April 23rd at 2:00 PM?
Based on your current calendar and To-Do list, I have analyzed your tasks and their respective due dates. I have proposed a schedule for you, which you can review and confirm.

Here's the proposed schedule for your tasks:

* April 18th, 2024:
	+ 14:00 - 15:00: Practice presentation
	+ 15:00 - 16:00: Finish setting up tasks API
	+ 16:00 - 17:00: 545 poster
* April 19th, 2024:
	+ 10:00 - 12:00: Finish EECS 583 Slides
	+ 14:00 - 15:00: Finish setting up t

In [49]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text. If there is no action required then output an empty string.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, assume default duration as 1 hour for any event. Use createEvent for scheduling tasks on the to-do list.
Available function templates to include in the response string:
- createEvent(description, date, time, duration)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_color, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, start_date, time, duration, frequency, until_date). Frequency can have the value of DAILY, WEEKLY, MONTHLY, YEARLY.
- deleteRecurringEvent(recurring_event_id)
- updateRecurringEvent(recurring_event_id, new_color, new_duration, new_location, new_frequency, new_until)
For all functions other than createEvent and createRecurringEvent, make sure to fill in the id of the event you are referring to from the calendar provided.
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Example input1: Schedule a team meeting for Friday afternoon at 3pm
Example output1: createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1:00)
Example input2: Update my team meeting to be two hours
Example output2: updateEvent("event_id", '', 2:00, '')
Example input3: Create a recurring meeting every Wednesday at 2pm until the end of June
Example output3: createRecurringEvent("Meeting", "YYYY-MM-DD", "02:00", "1:00", "WEEKLY", "YYYY-MM-DD")
Example input4: Update my recurring meeting every Wednesday at 2pm to be every month until the end of August
Example output4: updateRecurringEvent("RecurringEventId", "", "02:00", "1:00", "", "MONTHLY", "YYYY-MM-DD")
Example input5: Move my team meeting to tomorrow at 12pm
Example output5: moveEvent("event_id", 'YYYY-MM-DD', '12:00')
Example input6: Clear my afternoon
Example output7: deleteEvent("event_id");deleteEvent("event_id")
Example input8: Schedule all my tasks
Example output8: createEvent("Task 1", "YYYY-MM-DD", "10:00", 1:00);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1:00);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1:00)\n

Now, please generate the necessary function call(s) based on the user's request."""

In [50]:
messages.append(ChatMessage(
            role="system",
            content=instruction2)
        )
log_activity("system", instruction2)

In [51]:
function_call = ''
function_call = client.chat(model="mistral-large-latest", messages=messages)
function_call=function_call.choices[0].message.content
print(function_call)



createEvent("Meeting", "2024-04-18", "09:00", 1:00)


In [52]:
# Assuming the initial string of function calls is contained in a variable named function_call
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
    "updateEvent": methods.updateEvent,
    "createRecurringEvent": methods.createRecurringEvent,
    "updateRecurringEvent": methods.updateRecurringEvent,
    "deleteRecurringEvent": methods.deleteRecurringEvent,
    "inviteEmail": methods.inviteEmail,
    "moveRecurringEvent": methods.moveRecurringEvent,
}

# Iterate through the function calls
for call in function_calls:
    # Properly parse the call to extract function name and arguments
    if "(" in call and ")" in call:
        function_name, arguments = call.split('(', 1)
        arguments = arguments.rsplit(')', 1)[0].split(', ')

        # Call the corresponding function with the provided arguments
        if function_name in functions:
            functions[function_name](*arguments)
        else:
            print(f"Function '{function_name}' not found.")



['createEvent("Meeting", "2024-04-18", "09:00", 1:00)']
Create event called
The data and time is  2024-04-18 / 09:00
UTC start time:  2024-04-18 13:00:00+00:00 UTC end time:  2024-04-18 14:00:00+00:00
Event created: Meeting on 2024-04-18 at 09:00 for duration of 1:00 hours.
